In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comments-data/train.csv


# Using BERT 

### Loading Libraries

In [11]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split

### Load Data Set

In [12]:
df = pd.read_csv('/kaggle/input/comments-data/train.csv')

### Splitting into train and test data

In [13]:
train_data = df.drop(columns = ['id'])
X = train_data['comment_text'].to_list() # to_list becasue we want tokensize 
y = train_data.drop(columns = 'comment_text').values # returns a 2 D array each row is a row of labels each column represents a label

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

### Tokenizing input data

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_data(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf", max_length=max_length)
    
train_encodings = tokenize_data(train_X, tokenizer)
test_encodings = tokenize_data(test_X, tokenizer)

In [21]:
train_encodings['input_ids']

<tf.Tensor: shape=(127656, 128), dtype=int32, numpy=
array([[  101, 13055, 26568, ...,     0,     0,     0],
       [  101,  1010,  1023, ...,     0,     0,     0],
       [  101,  1000,  1996, ...,  1006,  1037,   102],
       ...,
       [  101,  1045,  2180, ...,     0,     0,     0],
       [  101,  1000,  2329, ...,     0,     0,     0],
       [  101,  2017,  2024, ...,     0,     0,     0]], dtype=int32)>

### Load and Compile Pretrained BERT Model

In [15]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6) 
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

bert_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Train the Model

In [ ]:
bert_model.fit(train_encodings['input_ids'], train_y, epochs=3, batch_size=16, validation_data=(test_encodings['input_ids'], test_y))

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1729362482.022773      78 service.cc:145] XLA service 0x7a8c38535cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729362482.022823      78 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729362482.022827      78 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1729362482.199776      78 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1458/7979 [====>.........................] - ETA: 44:10 - loss: 0.0718 - accuracy: 0.9818

In [ ]:
model.save('bert.h5')

### Make Predictions

In [ ]:
predictions = bert_model.predict(test_encodings['input_ids'])
binary_predictions = (tf.nn.sigmoid(predictions.logits).numpy() > 0.5).astype(int)

### Model Evaluation 

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy